In [1]:
import numpy as np
import tensorflow as tf

/Users/andyxie/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# 介绍

# 例子：线性回归

In [2]:
# 我们将重新使用estimator做一次线性回归，这一次我们还有了一组test数据
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_test = np.array([2., 5., 8., 1.])
y_test = np.array([-1.01, -4.1, -7, 0.])

In [7]:
# 首先我们需要定义输入量，X, 这里只有一个值，所以shape=[1]
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]
# 线性回归模型TF已经准备好了，在estimator下就有
estimator = tf.estimator.LinearRegressor(feature_columns)

# 下面将数值（x_train，y_train)传入模型，首先需要定义一个函数
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
# train函数将自动进行梯度下降操作，实际上estimator的内部工作机制更加复杂，在过程中还会有输出，并自动存储模型参数
estimator.train(input_fn=input_fn, steps=1000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/fz/y59hb15j1b1_ny8tfkvcj03000qswk/T/tmphljvwh1i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x181740cf28>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/fz/y59hb15j1b1_ny8tfkvcj03000qswk/T/tmphljvwh1i/model.ckpt.
INFO:tensorflow:loss = 17.0, step = 1
INFO:tensorflow:global_step/sec: 1656.7
INFO:tensorflow:loss = 0.0957528, step = 101 (0.062 sec)
INFO:tensorflow:global_step/sec: 2205.51
INFO:tensorflow:loss = 0.0974099, step =

In [10]:
# 如果想计算test数据集的相关数据，语法和上面也类似
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_test}, y_test, batch_size=4, num_epochs=1000, shuffle=False)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-11-22-05:40:18
INFO:tensorflow:Restoring parameters from /var/folders/fz/y59hb15j1b1_ny8tfkvcj03000qswk/T/tmphljvwh1i/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-11-22-05:40:19
INFO:tensorflow:Saving dict for global step 2000: average_loss = 4.66294e-13, global_step = 2000, loss = 1.86517e-12
INFO:tensorflow:Starting evaluation at 2017-11-22-05:40:19
INFO:tensorflow:Restoring parameters from /var/folders/fz/y59hb15j1b1_ny8tfkvcj03000qswk/T/tmphljvwh1i/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-11-22-05:40:19
INFO:tensorflow:Saving dict for global step 2000: average_loss = 0.00252507, global_step = 2000, loss = 0.0101003
train metrics: {'average_loss': 4.6629367e-13, 'loss': 1.8651747e-12, 'global_step': 2000}
eval metrics: {'average_loss': 0.0025250721, 'loss': 0.010100288, 'global_step': 2000}


In [ ]:
# 最后，如果要使用
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)